<img src="banner.png" width=500 style="margin-left:0; margin-right:auto; padding: 20px"/>

This notebook provides an introduction to ExoMDN, a machine-learning based model for the rapid characterization of exoplanet interiors. ExoMDN is based on Mixture Density Networks (MDNs), which output a mixture of Gaussian functions in order to approximate the distribution of interior structures which fit e.g. observed planet mass and planet radius.

For more details, see Baumeister and Tosi 2023

Contact: <philipp.baumeister@dlr.de>

In [ ]:
from exomdn import ExoMDN
from exomdn.plotting import cornerplot, cornerplot_logratios

# Setting up
</br>
Let's start by creating a new <code>ExoMDN</code> object. <code>ExoMDN</code> handles the MDN models and includes interactive widgets to facilitate loading models and running predictions.

In [ ]:
exo = ExoMDN(model_path="./models", data_path="./data")

# Loading a trained model
</br>
Next, we need to load a trained MDN model which we want to use for the interior prediction. To simplify things, we will use the included <code>load_model_widget</code>, which allows to interactively select which model to load. By default, <code>ExoMDN</code> searches for models in the  <i>./models</i> path. This can be changed by setting <code>exomdn.model_path</code>

By default, two models are available:

* ***mass_radius_Teq*** (takes planet mass, radius, and equilibrium temperature as inputs)
* ***mass_radius_k2_Teq*** (takes planet mass, radius, fluid Love number $k_2$ and equilibrium temperature as inputs)

In [ ]:
exo.load_model_widget

# Making an interior prediction
</br>
<code>ExoMDN</code> provides a custom widget to run a prediction for a single planet. The output of the MDN is in terms of a distribution of log-ratios of the mass and radius fractions of each interior layer of the planet. To convert to mass and radius fractions, the model samples from the distribution and transforms each point. The number of samples can be specified in the "Options" section of the widget.
</br>
</br>
Uncertainties can be included by ticking the checkbox in "Planet parameters". The model then first samples a number of times from within the error bars (how often can be set with the "Uncertainty samples" option) and predicts an interior distribution from each. From each of these predictions a number of points is then sampled so that the total number fits as closely as possible to the specified total number of samples (e.g. with the default values of 10 000 total samples and 1000 uncertainty samples, the model predicts 1000 distributions from within the error bars and then samples 10 times from each predicted distribution to get to the total of 10 000)

In [ ]:
exo.prediction_widget

The output of the prediction widget is saved in <code>ExoMDN</code> in the form of [pandas DataFrames](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) as follows:

* `exomdn.input_prompt` contains the input(s) to the prediction
* `exomdn.prediction` contains the predicted samples of the interior
* `exomdn.mixture_components` contains the means, variances, and mixture weights of the predicted Gaussian mixture

In [ ]:
print(f"Length of input: {len(exo.input_prompt)}")
print(f"Number of mixture components: {len(exo.mixture_components)}")
print("=" * 40)
print("Prediction Summary:")
exo.prediction.describe()

In [ ]:
exo.mixture_components

# Visualization

We can visualize the output of the MDN with the `cornerplot_logratios` function. It takes as input the prediction data, the mixture components, and the log-ratio data columns one wants to visualize (' (`exo.rf_logratios` for radius fractions, `exo.mf_logratios` for mass fractions, `exo.logratios` for both).

The upper right plots also show the distribution of Gaussian kernels as predicted by the MDN, where the colors mark the respective weight in the distribution.

In [ ]:
# showing radius fractions
cornerplot_logratios(data=exo.prediction, data_components=exo.mixture_components, columns=exo.rf_logratios, height=2)

# showing mass fractions
# cornerplot_logratios(data=exomdn.prediction, data_components=exomdn.mixture_components, columns=exomdn.mf_logratios, height=2)

# showing both radius and mass fractions
# cornerplot_logratios(data=exomdn.prediction, data_components=exomdn.mixture_components, columns=exomdn.logratios, height=1.5)

The `cornerplot` function can be used to show the predicted interior in terms of true radius and mass fractions instead of log-ratios.

In [ ]:
# showing radius fractions
cornerplot(data=exo.prediction, columns=exo.rf, height=2)

# showing mass fractions
# cornerplot(data=exomdn.prediction, columns=exomdn.mf, height=2)